In [1]:
%pip install pandas numpy scikit-learn nltk matplotlib seaborn joblib imbalanced-learn

  Using cached imbalanced_learn-0.14.0-py3-none-any.whl.metadata (8.8 kB)
Using cached imbalanced_learn-0.14.0-py3-none-any.whl (239 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\vvena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip uninstall -y scikit-learn
%pip install scikit-learn==1.4.2

Found existing installation: scikit-learn 1.4.2
Uninstalling scikit-learn-1.4.2:
  Successfully uninstalled scikit-learn-1.4.2
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.4.2-cp311-cp311-win_amd64.whl (10.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\vvena\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Baixar dataset 
utilize este link: [dataset](https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets). 

Vá até o concatenated e baixe-o  

In [ ]:
import re
import joblib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from imblearn.pipeline import Pipeline

from tempfile import mkdtemp
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns


In [5]:
df = pd.read_csv("./datasets/concatenated.csv")


C:\Users\vvena\AppData\Local\Temp\ipykernel_19280\178527699.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./datasets/concatenated.csv")


In [6]:
df.drop(columns=["dataset", "original_index", "review_text_processed", "review_text_tokenized", "rating", "kfold_polarity", "kfold_rating"], inplace=True)
df.head()

,review_text,polarity
0,Bem macio e felpudo...recomendo. Preço imbatí...,1.0
1,Produto excepcional! recomendo!!! inovador e ...,1.0
2,recebi o produto antes do prazo mas veio com d...,0.0
3,Bom custo beneficio. Adequado para pessoas que...,1.0
4,Além de higiênico tem o tamanho ideal. Só falt...,NaN


In [7]:
df["polarity"].replace({1: "positivo", 0: "negativo", np.nan: "neutro"}, inplace=True)
df["sentimento"] = df["polarity"]
df.drop(columns=["polarity"], inplace=True)
df.head()

C:\Users\vvena\AppData\Local\Temp\ipykernel_19280\2086818870.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["polarity"].replace({1: "positivo", 0: "negativo", np.nan: "neutro"}, inplace=True)


,review_text,sentimento
0,Bem macio e felpudo...recomendo. Preço imbatí...,positivo
1,Produto excepcional! recomendo!!! inovador e ...,positivo
2,recebi o produto antes do prazo mas veio com d...,negativo
3,Bom custo beneficio. Adequado para pessoas que...,positivo
4,Além de higiênico tem o tamanho ideal. Só falt...,neutro


In [8]:
df['sentimento'].value_counts()

sentimento
positivo    1976535
negativo     409629
neutro       399928
Name: count, dtype: int64

In [9]:
def clean_text(text):
    text = str(text).lower()
    text = text.replace("lojas americanas", "loja")
    text = text.replace("americanas", "loja")
    text = text.replace("americana", "loja")
    text = re.sub(r"r?$ ?\d+([.,]\d+)?", "dinheiro", text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\S+", "", text)
    text = re.sub(r"[^a-z0-9à-úç ]", "", text)
    text = " ".join(text.split())
    return text

nltk.download("stopwords")
stop_pt = stopwords.words("portuguese")

df.dropna(subset=['review_text', 'sentimento'], inplace=True)
df['review_text'] = df['review_text'].astype(str).apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vvena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
x = df["review_text"]
y = df["sentimento"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(f"Treino: {len(X_train)} amostras | Teste: {len(X_test)} amostras")

Treino: 2228872 amostras | Teste: 557219 amostras


# Modelo Base com Logistic Regression  

In [15]:
pipeline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(stop_words=stopwords.words("portuguese"))),
        ("clf", LogisticRegression(class_weight="balanced", solver="saga")),
    ]
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.4f}")

Acurácia: 0.6991


# Modelo com GridSearch e dataset Inteiro


In [16]:
param_grid = {
    'tfidf__ngram_range': [(1,2)],
    'clf__C': [1, 10],
    'clf__penalty': ['l2']
}

grid_search = GridSearchCV(
    pipeline,
    param_grid = param_grid,
    cv = 3,
    n_jobs = -1, 
    verbose = 1
)

In [17]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


C:\Users\vvena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['a', 'à',
                                                                    'ao', 'aos',
                                                                    'aquela',
                                                                    'aquelas',
                                                                    'aquele',
                                                                    'aqueles',
                                                                    'aquilo',
                                                                    'as', 'às',
                                                                    'até',
                                                                    'com',
                                                                    'como',
                                                                    'da', 'das',
                                                                    'de',
                                                                    'dela',
                                                                    'delas',
                                                                    'dele',
                                                                    'deles',
                                                                    'depois',
                                                                    'do', 'dos',
                                                                    'e', 'é',
                                                                    'ela',
                                                                    'elas',
                                                                    'ele',
                                                                    'eles', ...])),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           solver='saga'))]),
             n_jobs=-1,
             param_grid={'clf__C': [1, 10], 'clf__penalty': ['l2'],
                         'tfidf__ngram_range': [(1, 2)]},
             verbose=1)

In [18]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("classification_report:", classification_report(y_test, y_pred))

classification_report:               precision    recall  f1-score   support

    negativo       0.63      0.70      0.66     81442
      neutro       0.36      0.39      0.38     79966
    positivo       0.88      0.85      0.86    395811

    accuracy                           0.76    557219
   macro avg       0.62      0.65      0.63    557219
weighted avg       0.77      0.76      0.76    557219



# Modelo base Com uma amostra do dataset

In [ ]:
df_small = (
    df
    .groupby("sentimento", group_keys=False)
    .apply(lambda x: x.sample(200_000, random_state=42))
)


C:\Users\vvena\AppData\Local\Temp\ipykernel_19280\2044130809.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(200_000, random_state=42))


review_text  sentimento
False        False         600000
Name: count, dtype: int64

In [54]:
df_small["sentimento"].value_counts()

sentimento
negativo    200000
neutro      200000
positivo    200000
Name: count, dtype: int64

In [52]:
x = df_small["review_text"]
y = df_small["sentimento"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(f"Amostras usadas para treinamento: {len(x_train)}")


Amostras usadas para treinamento: 480000


In [53]:
pipeline.fit(x_train, y_train)

y_predict = pipeline.predict(x_test)

print("Classification Report:", classification_report(y_test, y_predict))

Classification Report:               precision    recall  f1-score   support

    negativo       0.73      0.72      0.73     39964
      neutro       0.56      0.56      0.56     40099
    positivo       0.69      0.70      0.69     39937

    accuracy                           0.66    120000
   macro avg       0.66      0.66      0.66    120000
weighted avg       0.66      0.66      0.66    120000



# Modelo com amostra usando gridsearch

In [58]:
param_grid = {
    "tfidf__ngram_range": [(1,2)],
    "clf__C": [1, 10],
    "clf__penalty": ["l2"]
}

grid_search = GridSearchCV(
    pipeline, 
    param_grid = param_grid,
    scoring = "balanced_accuracy",
    cv = 3,
    n_jobs = -1,
    verbose = 2 
)

In [59]:
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['a', 'à',
                                                                    'ao', 'aos',
                                                                    'aquela',
                                                                    'aquelas',
                                                                    'aquele',
                                                                    'aqueles',
                                                                    'aquilo',
                                                                    'as', 'às',
                                                                    'até',
                                                                    'com',
                                                                    'como',
                                                                    'da', 'das',
                                                                    'de',
                                                                    'dela',
                                                                    'delas',
                                                                    'dele',
                                                                    'deles',
                                                                    'depois',
                                                                    'do', 'dos',
                                                                    'e', 'é',
                                                                    'ela',
                                                                    'elas',
                                                                    'ele',
                                                                    'eles', ...])),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           solver='saga'))]),
             n_jobs=-1,
             param_grid={'clf__C': [1, 10], 'clf__penalty': ['l2'],
                         'tfidf__ngram_range': [(1, 2)]},
             scoring='balanced_accuracy', verbose=2)

In [60]:
best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test)

print(f"Classification report: {classification_report(y_test, y_pred)}")

Classification report:               precision    recall  f1-score   support

    negativo       0.74      0.74      0.74     39964
      neutro       0.57      0.57      0.57     40099
    positivo       0.70      0.71      0.70     39937

    accuracy                           0.67    120000
   macro avg       0.67      0.67      0.67    120000
weighted avg       0.67      0.67      0.67    120000



In [74]:
dict_comp = {
    "texto": x_test,
    "sentimento_verdadeiro": y_test,
    "sentimento_previsto": y_pred
}

df_comp = pd.DataFrame(dict_comp)
df_comp.head()

,texto,sentimento_verdadeiro,sentimento_previsto
1253265,inland empire é um filme fraco do david lynch diferente de mulholland dr 2001 ou lost highway 1997 nos quais muitos elementos aparentemente aleatóriosacabam se correlacionando e adquirindo sentido inland empire me parece ser um mero disperso e dispensável exercício estilista desnecessário levando em conta os filmes anterioresdo diretor que não estimula o espectador a pensar ou refletir sobre muita coisa,negativo,neutro
13055,não comprem esse produto não manteve carga nem sequer com 2 dias de uso demorando 10 horas para carregar pior impossível,negativo,negativo
1778216,vou ter que sair atrás desse filme pois dificilmente estreará aqui em sorocaba,neutro,neutro
179369,nao gostei deste por que ele fica sem rede eu no facebook mais ele nao ta ativo sempre aparece procurando rede,negativo,negativo
2386354,muito lindo grandes composições só pecou pelo roteiro,positivo,neutro


In [79]:
df_comp.query("(sentimento_verdadeiro != sentimento_previsto) & (sentimento_previsto == 'neutro')")

,texto,sentimento_verdadeiro,sentimento_previsto
1253265,inland empire é um filme fraco do david lynch diferente de mulholland dr 2001 ou lost highway 1997 nos quais muitos elementos aparentemente aleatóriosacabam se correlacionando e adquirindo sentido inland empire me parece ser um mero disperso e dispensável exercício estilista desnecessário levando em conta os filmes anterioresdo diretor que não estimula o espectador a pensar ou refletir sobre muita coisa,negativo,neutro
2386354,muito lindo grandes composições só pecou pelo roteiro,positivo,neutro
1541995,bom eu acabei baixando o filme pq eu tenho receio em gastar dinheiro pra ir ao cinema ver musical são raríssimos os que me agradam então fui assistir la la land já com um leve sentimento de ódio por ser musical e por ter levado tantos globos de ouro por ser um musical haha e nossa me ganhou na primeira cena com aquele clima tão nostálgico emma stone está fantástica no filme torço pra ela no oscar pra continuar se empenhando e fazendo ótimos papéis e o ryan gosling está muito bom também nada uau e que justifique prêmios colin farrell por exemplo merecia mais o gg mas a química entre os dois foi linda de se ver enfim me surpreendi bastante e acredito que merece tudo que está colhendo,positivo,neutro
2457368,tirando o roteiro um pouco clichê é engraçado divertido e possui excelentes cenas de luta a cena do bar e da igreja foram as melhores cenas de 2014 na minha opinião tirou aquele esteriótipo que espião precisa ser totalmente sério ótimo filme,positivo,neutro
1907666,30122018 nota 8 a ideia é incrivel noomi maravilhosa o filme me prendeu do inicio ao quase final final cagou muito decepcionante,positivo,neutro
...,...,...,...
1179845,lógico que eu vou assistir o filme a chloe moretz é jovem mas da de 10 à 0 em muita marmanja nos cinemas aí em particular minha teima é por ser o que a terceira versão de carrie sei lá cadê novos projetos parece que os reis do cinema não se importam mais em criar são só remakes e continuações sabe a gente sempre espera algo novo mas é isso oque está tendo acho que seja desnecessário sim uma outra versão mas jamais que eu vou deixar de assistir por conta disso ansiosa pra ver a chloe surpreendendo nos cinemas de novo na minha opinião chloe moretz e elle fanning são as garotas que vão salvar essa nossa geração do cinema daqui pra frente,negativo,neutro
645308,as traduções em espanhol não estão todas certas,negativo,neutro
935164,um lixão por favor arrumem e facilitem a nossa vida o que vocês acham fica a dica,negativo,neutro
1858935,sessão da tarde é você o0compreendi sim o filme mas ainda assim o achei muitíssimo fraco inclusive no texto dos personagens quando parecia que ficaria bomo filme acaba sinceramente na minha humilde opinião deixou a desejar inclusive pra uma possível filmagem de chappie 2 será o0,negativo,neutro


# Modelo utilizando apenas 2 tipos de classificação

In [114]:
df_small_2= df_small[df_small["sentimento"] != "neutro"]

df_small_2.head()

,review_text,sentimento
1167046,o app até baixa mas um conteúdo bem resumido e outra dps de baixar no cartão sd começa a baixar de novo gastando os dados móveis e enchendo a memória que lixoooo minha internet acabando todo mês por conta disso tô fora,negativo
1448452,não consegui gostar desse filme,negativo
253723,a programação ao vivo não carrega de jeito nenhum uma bosta,negativo
154260,péssimo app não abre maior burocracia para fazer o cadastro da o maior erro e ainda no fim não temos acesso pq da erro novamente,negativo
467281,não consigo editar meu video sempre para e volta do começo para recomeçar tudo de novo,negativo


In [116]:
x_2 = df_small_2["review_text"]
y_2 = df_small_2["sentimento"]


x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_2, y_2, test_size=0.2, random_state=42)

In [ ]:
pipeline.fit()

In [117]:
grid_search_2 = GridSearchCV(
    pipeline,
    param_grid= param_grid,
    scoring = "balanced_accuracy",
    cv =3,
    n_jobs = -1,
    verbose=3
)

grid_search_2.fit(x_train_2, y_train_2)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['a', 'à',
                                                                    'ao', 'aos',
                                                                    'aquela',
                                                                    'aquelas',
                                                                    'aquele',
                                                                    'aqueles',
                                                                    'aquilo',
                                                                    'as', 'às',
                                                                    'até',
                                                                    'com',
                                                                    'como',
                                                                    'da', 'das',
                                                                    'de',
                                                                    'dela',
                                                                    'delas',
                                                                    'dele',
                                                                    'deles',
                                                                    'depois',
                                                                    'do', 'dos',
                                                                    'e', 'é',
                                                                    'ela',
                                                                    'elas',
                                                                    'ele',
                                                                    'eles', ...])),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           solver='saga'))]),
             n_jobs=-1,
             param_grid={'clf__C': [1, 10], 'clf__penalty': ['l2'],
                         'tfidf__ngram_range': [(1, 2)]},
             scoring='balanced_accuracy', verbose=3)

In [120]:
best_model = grid_search_2.best_estimator_
y_pred_2 = best_model.predict(x_test_2)

print(f"Classification report: {classification_report(y_test_2, y_pred_2)}")

Classification report:               precision    recall  f1-score   support

    negativo       0.87      0.87      0.87     40051
    positivo       0.87      0.87      0.87     39949

    accuracy                           0.87     80000
   macro avg       0.87      0.87      0.87     80000
weighted avg       0.87      0.87      0.87     80000



# Mudando params 

In [135]:
pipeline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(stop_words=stopwords.words("portuguese"))),
        ("clf", LogisticRegression(solver="sag")),
    ]
)

param_grid_2 = {
    "tfidf__ngram_range": [(1,2)],
    "clf__C": [1, 10],
    "clf__penalty": ["l2"]
}

grid_search_2 = GridSearchCV(
    pipeline,
    param_grid=param_grid_2,
    scoring="balanced_accuracy",
    cv=3,
    n_jobs= -1
)

grid_search_2.fit(x_train_2, y_train_2)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words=['a', 'à',
                                                                    'ao', 'aos',
                                                                    'aquela',
                                                                    'aquelas',
                                                                    'aquele',
                                                                    'aqueles',
                                                                    'aquilo',
                                                                    'as', 'às',
                                                                    'até',
                                                                    'com',
                                                                    'como',
                                                                    'da', 'das',
                                                                    'de',
                                                                    'dela',
                                                                    'delas',
                                                                    'dele',
                                                                    'deles',
                                                                    'depois',
                                                                    'do', 'dos',
                                                                    'e', 'é',
                                                                    'ela',
                                                                    'elas',
                                                                    'ele',
                                                                    'eles', ...])),
                                       ('clf',
                                        LogisticRegression(solver='sag'))]),
             n_jobs=-1,
             param_grid={'clf__C': [1, 10], 'clf__penalty': ['l2'],
                         'tfidf__ngram_range': [(1, 2)]},
             scoring='balanced_accuracy')

In [136]:
best_model = grid_search_2.best_estimator_

y_pred_2 = best_model.predict(x_test_2)

print(f"Classification report: {classification_report(y_test_2, y_pred_2)}")

Classification report:               precision    recall  f1-score   support

    negativo       0.87      0.87      0.87     40051
    positivo       0.87      0.87      0.87     39949

    accuracy                           0.87     80000
   macro avg       0.87      0.87      0.87     80000
weighted avg       0.87      0.87      0.87     80000

